In [ ]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
import numpy as np
import pandas as pd


In [ ]:
input(ACCESS_TOKEN)
HEADERS = {"Authorization": "Bearer " + ACCESS_TOKEN}
TRANSPORT = RequestsHTTPTransport(
    url="https://api.yelp.com/v3/graphql", headers=HEADERS)
CLIENT = Client(transport=TRANSPORT, fetch_schema_from_transport=True)


In [ ]:

def get_restaurants(categories, location, radius):
    query = gql(
        """
      {
        search(categories: "%s",
              location: "%s"
              radius: %d) {
          total
        }
      }
  """
        % (categories, location, np.minimum(radius, 40_000)))

    return CLIENT.execute(query)


In [ ]:
df = pd.read_csv("../cities.csv", index_col=[0, 1])

data = pd.DataFrame(index=df.index)
data["population"] = df["2020 census"]
data["area"] = df["2020 land area"]

# convert from sq mi to sq m, a = pi*r^2
data["radius"] = np.sqrt(data["area"]*2589988.1103/np.pi)

data.tail()

In [ ]:
data["restaurants"] = 0

for i, row in data.iterrows():
    location = "%s, %s" % (i[0], i[1])
    # print(location)
    try:
        result = get_restaurants("vegan", location, row["radius"])
        data.loc[i, "restaurants"] = int(result["search"]["total"])
    except Exception as e:
        print(e)


In [ ]:
vegan = pd.read_csv("vegan.csv", index_col=[0, 1])
data.loc[("Asheville", "North Carolina"), "restaurants"] = 4
data = vegan.append(data.loc[("Asheville", "North Carolina")])

data["per_100k"] = data["restaurants"] / data["population"] * 100_000

chicago = data.loc[("Chicago", "Illinois"), "per_100k"]
data["score"] = np.log2(data["per_100k"] / chicago)

data.sort_values("score", ascending=False, inplace=True)


In [ ]:
print(data.to_string())
data.to_csv("vegan.csv")
